**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


## Import data

In [2]:
ASSAY    = "KS91_K562_ASTARRseq"
FOLDER   = "coverage_astarrseq_peak_macs_input"

In [3]:
fdiry = file.path(FD_RES, "results", ASSAY, FOLDER, "summary")
fname = "result.Log2FC.raw.deseq.WGS.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_log2fc = dat
print(dim(dat))
head(dat)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.WGS.tsv"
[1] 246850      7


Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1_10015_10442,71.40375,-2.2136128,0.2691517,-8.2244061,1.961605e-16,2.910688e-15
chr1_14253_14645,68.63035,0.2221859,0.1534101,1.4483134,1.475294e-01,2.178106e-01
chr1_16015_16477,71.25409,-1.2712950,0.2019628,-6.2946978,3.079993e-10,2.653010e-09
chr1_17237_17772,124.03128,-1.3852891,0.1638628,-8.4539594,2.815873e-17,4.431893e-16
chr1_28903_29613,113.63103,-1.9422003,0.1886131,-10.2972733,7.248681e-25,1.723665e-23
chr1_30803_31072,57.37875,-0.1332069,0.2145692,-0.6208109,5.347240e-01,6.231400e-01


In [4]:
fdiry = file.path(FD_RES, "results", "comparison")
fname = "dat_peaks_info.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

###
dat_peak_info = dat
dat_peak      = dat %>% dplyr::select(Peak, Dataset, Region)
print(dim(dat))
head(dat)

[1] 1451    6


Peak,Chrom,Start,End,Dataset,Region
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr11_61792068_61793464,chr11,61792068,61793464,OL13,FADS
chr11_61800085_61801113,chr11,61800085,61801113,OL13,FADS
chr11_61806630_61807154,chr11,61806630,61807154,OL13,FADS
chr11_61814735_61817343,chr11,61814735,61817343,OL13,FADS
chr11_61822094_61822443,chr11,61822094,61822443,OL13,FADS
chr11_61825795_61826306,chr11,61825795,61826306,OL13,FADS


## Rank normalization

In [5]:
fun_ranknorm = function(x){
    return(rank(x)/length(x))
}

In [6]:
dat = left_join(dat_peak, dat_log2fc, by="Peak")

dat = dat %>%
    dplyr::group_by(Region) %>%
    dplyr::mutate(RankNorm_Region  = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::group_by(Dataset) %>%
    dplyr::mutate(RankNorm_Dataset = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::mutate(RankNorm_Total  = fun_ranknorm(log2FoldChange))

dat_ranknorm = dat
print(dim(dat))
tail(dat)

[1] 1451   12


Peak,Dataset,Region,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,RankNorm_Region,RankNorm_Dataset,RankNorm_Total
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr2_61524551_61524988,OL45,BCL11A,86.96814,0.9884293,0.1778294,5.558302,2.724117e-08,1.837642e-07,0.8315217,0.9449275,0.9434873
chr2_61529854_61531204,OL45,BCL11A,291.41614,0.7922681,0.1169114,6.776653,1.229914e-11,1.231161e-10,0.6956522,0.9091787,0.9145417
chr2_61536312_61537087,OL45,BCL11A,1715.53062,1.1080431,0.1827582,6.062890,1.336972e-09,1.068478e-08,0.8913043,0.9594203,0.9579600
chr2_61537852_61539255,OL45,BCL11A,9063.07637,1.3395927,0.2243423,5.971199,2.355159e-09,1.827061e-08,0.9565217,0.9777778,0.9738112
chr2_61545554_61545869,OL45,BCL11A,62.54110,0.9184583,0.2287748,4.014684,5.952555e-05,2.254251e-04,0.7826087,0.9333333,0.9338387
chr2_61549710_61550633,OL45,BCL11A,119.85652,0.6213305,0.1888286,3.290446,1.000286e-03,2.841500e-03,0.5597826,0.8685990,0.8780152


In [7]:
fdiry = file.path(FD_RES, "results", "comparison", "comparison_global")
fname = "result.ranknorm.astarr.tsv"
fpath = file.path(fdiry, fname)

dat = dat_ranknorm
write_tsv(dat, fpath)